<br><br><center><h1 style="font-size:4em;color:#2467C0">Does increase in crime endanger tourism in India?</h1></center>


Tourism in India is economically important and is growing rapidly. The World Travel & Tourism Council calculated that tourism generated ₹14.02 lakh crore (USD 220 billion) or 9.6\% of the nation's GDP in 2016 and supported 40.343 million jobs, 9.3\% of its total employment. 
<br>

The sector is predicted to grow at an annual rate of 6.8% to ₹28.49 lakh crore (USD 440 billion) by 2027 (10% of GDP). In October 2015, India's medical tourism sector was estimated to be worth USD 3 billion. It is projected to grow to USD 7–8 billion by 2020. In 2014, 184,298 foreign patients traveled to India to seek medical treatment.
<br>

Recently, India is named in the list of "unsafe" countries for Women travelers and also generally for tourists. Here is the link to the article 

<a href=https://www.forbes.com/sites/laurabegleybloom/2017/07/28/10-most-dangerous-places-for-women-travelers-and-how-to-stay-safe/#2b4872eb2448> Follow the link here </a>
<br>

<B> Dataset Credits </B>
<br>
    <B> - </B> Main Dataset - https://www.kaggle.com/rajanand/crime-in-india
    <br>
    <B> - </B> Supporting datasets (trends, pie charts) - https://en.wikipedia.org/wiki/Tourism_in_India
    <br>
    <B> - </B> Introduction Source - https://en.wikipedia.org/wiki/Tourism_in_India
    <br>
     <br>
<B> Visualization Credits </B>
 <br>
    <B> - </B> Plotly - https://plot.ly
     <br>
    <B> - </B> Mapbox - https://www.mapbox.com/
     <br>
</html>



In [69]:
import pandas as pd
import requests
import os
import pickle
import matplotlib.pyplot as plt
import plotly 
import plotly.plotly as py
from plotly.graph_objs import *
mapbox_access_token = ''

In [72]:
if not os.path.exists('plotly_secret.pk1'):
    plotly_creds={}
    plotly_creds['user']=''
    plotly_creds['api_key']=''
    plotly_creds['map_key']=''
    with open ('plotly_secret.pk1','wb') as f:
        pickle.dump(plotly_creds,f)
else:
    plotly_creds=pickle.load(open('plotly_secret.pk1', 'rb'))

In [25]:
plotly.tools.set_credentials_file(username=plotly_creds['user'], api_key=plotly_creds['api_key'])

In [26]:
# Let's see some trends and numbers to understand the motivation - Source (Wikipedia)

tour_trend = pd.read_csv('./crime-in-india/external/India_trourism_trend.csv',encoding='latin-1')
tour_trend.head()

,Year,Number (millions),æ% change
0,1997,2.37,3.8
1,1998,2.36,-0.7
2,1999,2.48,5.2
3,2000,2.65,6.7
4,2001,2.54,-4.2


In [28]:
# Plotting the trend using plotly

val_x = tour_trend.Year
val_y = tour_trend['Number (millions)']
trace = Scatter(
    x = val_x,
    y = val_y,
    mode = 'lines+markers',
    name = 'lines+markers')
data = [trace]
layout = dict(title='Average Number of Visitors to India - 1997 to 2011')
fig = dict(data=data, layout=layout)
#trace = Scatter(x,y)
py.iplot(fig,file_name='line')


<B> Average visitors is growing despite few dips in between. India has a culturally rich places, forts, temples, churches to visit. Unesco has almost 36 heritage sites across the country with 25 more pending to be approved. We will see some prominent tourist spots as we progress. </B>

In [29]:
# Source countries for foreign tourist arrivals in India : top 10 countries. Source (Wikipedia). 

tour_trend = pd.read_csv('./crime-in-india/external/Foreign_Visitors.csv',encoding='latin-1')
tour_trend.head()

,Country,Visitors
0,Bangladesh,1.380409
1,United States,1.296939
2,United Kingdom,0.941883
3,Canada,0.317239
4,Malaysia,0.301961


In [30]:
# Plotting the pie to visually see the number

labels = tour_trend.Country
values = tour_trend.Visitors
trace = Pie(labels=labels, values=values)
py.iplot([trace], title='basic_pie_chart')


<Center> <B> Bangladesh is very close to India and apparently is at top of the list. Chart is diverse and has countries from several continents </B><Center>

In [31]:
# Share of top 10 states/UTs of India in number of domestic tourist visits. Source (Wikipedia)

tour_rev = pd.read_csv('./crime-in-india/external/top_tourist_visiting_places.csv')
tour_rev.head()

,State,Tourists_in_Millions,Tourists
0,Tamil Nadu,333.459047,"333,459,047"
1,Uttar Pradesh,204.888457,"204,888,457"
2,Andhra Pradesh,121.591054,"121,591,054"
3,Karnataka,119.863942,"119,863,942"
4,Maharashtra,103.403934,"103,403,934"


In [53]:
# Plotting pie chart

labels = tour_rev.State
values = tour_rev.Tourists_in_Millions
trace = Pie(labels=labels, values=values)
py.iplot([trace], title='basic_pie_chart')


<Center> <B> Most of these places are covered in the bottom section, where we discuss about crime, tourists spots.  </B> <Center>

In [32]:
! ls ./crime-in-india/external

API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.zip
Chapter_26_Tourism-Table_26.2-Rankwise_Calculation.csv
Foreign_Visitors.csv
India_trourism_trend.csv
NO_OF_APPROVED_ACCOMODATION_AND_ROOMS_ALONG_WITH_THOSE_PENDING_FOR_CLASSIFICATION.xls
National_Parks_India copy.csv
National_Parks_India.csv
National_Parks_India.xlsx
Tourism_Statistics_Since_1991.xls
np_india.csv
np_lat_long.csv
rs_session-239_AU693_1.1.csv
state_ut_lat_lon copy.csv
state_ut_lat_lon.csv
tabula-police_person_table.csv
top_tourist_visiting_places.csv
whc-sites-2017.xls


In [33]:
### Let's start by identifying prominent tourist spots in India. 
# This list has UNESCO sites and other popular places from Wikipedia


National_Sites = pd.read_csv('./crime-in-india/external/np_india.csv', encoding='latin-1')
National_Sites.head()
National_Sites.shape

(111, 4)

In [34]:
# Function to get Latitude and Longitude locations for any given address or name. 
# this is resuable and we use it more than once as we progress.

def get_lat_long(xname, yname):
        temp_list = []
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + xname.lower() + "'")
        resp_json_payload = response.json()
        if ((resp_json_payload['status']) == 'OK'):
            temp_list = list(dict.values(resp_json_payload['results'][0]['geometry']['location']))
            temp_list.append(yname)
            return temp_list
        else:
            temp_list = []
            return temp_list



In [35]:
# So, we don't have latitude and longitude information for the tourist spots. 
# Let's create a document with spots and lat/long information. 

if os.path.exists('./crime-in-india/external/np_lat_long.csv'):
    os.remove('./crime-in-india/external/np_lat_long.csv')

temp_list = list(National_Sites['Name'])
with open('./crime-in-india/external/np_lat_long.csv', 'w') as f:
    f.write("lat,lon,place")
    f.write('\n')
    for i in range(len(temp_list)):
        temp_str = temp_list[i].replace(" ", "+")
        line = str(get_lat_long(temp_str, temp_list[i]))
        jk_line1 = line.replace("[","")
        jk_line2 = jk_line1.replace("]","")
        f.write(jk_line2)
        f.write('\n')


In [36]:
# Exploring the output document from above step. Now, we have lat/long information to plot.

tframe = pd.read_csv('./crime-in-india/external/np_lat_long.csv')
tframe.shape

(59, 3)

In [74]:
# Using Plotly, let's plot the tourist spots.
# We can zoom in and out to interact with the visual.

mapbox_access_token = plotly_creds['map_key']

df = pd.read_csv('./crime-in-india/external/np_lat_long.csv')
site_lat = df.lat
site_lon = df.lon
locations_name = df.place

data = Data([
    Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=Marker(
            size=12,
            color='rgb(255, 0, 0)',
            opacity=0.8
        ),
        text=locations_name,
        hoverinfo='text'
    )]
)
        
layout = Layout(
    title='Prominent Tourist Spots in India',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=22.6,
            lon=93.4
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='India Mapbox')

<B> So, this is vast and all over. The important thing is, these are not the only spots. Infact, we didn't imclude temples and churches in list Individually. But, those spots are covered in the data.   </B> 

<h5 style="font-size:2em;color:#2467C0">Identifying & Plotting Crime Indicators</h5>

In [38]:
crime_data = pd.read_csv('./crime-in-india/Crime_data/01_District_wise_crimes_committed_IPC_2001_2012.csv')
crime_data.head()

,STATE/UT,DISTRICT,YEAR,MURDER,ATTEMPT TO MURDER,CULPABLE HOMICIDE NOT AMOUNTING TO MURDER,RAPE,CUSTODIAL RAPE,OTHER RAPE,KIDNAPPING & ABDUCTION,...,ARSON,HURT/GREVIOUS HURT,DOWRY DEATHS,ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY,INSULT TO MODESTY OF WOMEN,CRUELTY BY HUSBAND OR HIS RELATIVES,IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES,CAUSING DEATH BY NEGLIGENCE,OTHER IPC CRIMES,TOTAL IPC CRIMES
0,ANDHRA PRADESH,ADILABAD,2001,101,60,17,50,0,50,46,...,30,1131,16,149,34,175,0,181,1518,4154
1,ANDHRA PRADESH,ANANTAPUR,2001,151,125,1,23,0,23,53,...,69,1543,7,118,24,154,0,270,754,4125
2,ANDHRA PRADESH,CHITTOOR,2001,101,57,2,27,0,27,59,...,38,2088,14,112,83,186,0,404,1262,5818
3,ANDHRA PRADESH,CUDDAPAH,2001,80,53,1,20,0,20,25,...,23,795,17,126,38,57,0,233,1181,3140
4,ANDHRA PRADESH,EAST GODAVARI,2001,82,67,1,23,0,23,49,...,41,1244,12,109,58,247,0,431,2313,6507


In [39]:
# Let's understand the data range

crime_data['YEAR'].unique().tolist()

[2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]

In [40]:
# Understanding the number of states. 

sample_state = crime_data['STATE/UT'].unique().tolist()
len(sample_state)

35

In [41]:
# Data is very granular and present at district level. 
# Grouping the data on State/UT level to understand the size of the data. 

crime_data[['STATE/UT', 'YEAR']].groupby('STATE/UT').count().sort_values('YEAR', ascending=False)

,YEAR
STATE/UT,
UTTAR PRADESH,878
MADHYA PRADESH,628
MAHARASHTRA,552
BIHAR,540
TAMIL NADU,469
RAJASTHAN,454
ODISHA,430
KARNATAKA,399
GUJARAT,384


In [42]:
# Every state has a district "Total" which actually indicates sum of values. 
# Here, we play around few filters to get a feel for the data we are dealing with. 

#filter = crime_data['STATE/UT'] == 'LAKSHADWEEP'
filter = (crime_data['STATE/UT'] == 'LAKSHADWEEP') & (crime_data['DISTRICT'] == 'TOTAL')
crime_data[filter]

,STATE/UT,DISTRICT,YEAR,MURDER,ATTEMPT TO MURDER,CULPABLE HOMICIDE NOT AMOUNTING TO MURDER,RAPE,CUSTODIAL RAPE,OTHER RAPE,KIDNAPPING & ABDUCTION,...,ARSON,HURT/GREVIOUS HURT,DOWRY DEATHS,ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY,INSULT TO MODESTY OF WOMEN,CRUELTY BY HUSBAND OR HIS RELATIVES,IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES,CAUSING DEATH BY NEGLIGENCE,OTHER IPC CRIMES,TOTAL IPC CRIMES
713,LAKSHADWEEP,TOTAL,2001,1,0,0,0,0,0,0,...,2,3,0,0,0,0,0,0,17,36
1432,LAKSHADWEEP,TOTAL,2002,0,0,0,1,0,1,0,...,2,5,0,1,0,0,0,0,5,53
2160,LAKSHADWEEP,TOTAL,2003,0,0,0,2,0,2,0,...,2,1,0,1,0,1,0,0,6,31
2889,LAKSHADWEEP,TOTAL,2004,0,0,0,0,0,0,0,...,0,6,0,0,0,1,0,0,18,70
3622,LAKSHADWEEP,TOTAL,2005,1,0,0,0,0,0,0,...,0,4,0,0,0,0,0,0,22,42
4361,LAKSHADWEEP,TOTAL,2006,0,2,0,0,0,0,0,...,4,5,0,0,0,1,0,0,37,80
5104,LAKSHADWEEP,TOTAL,2007,0,2,0,1,0,1,0,...,0,0,0,2,0,2,0,0,26,56
5865,LAKSHADWEEP,TOTAL,2008,1,0,0,2,0,2,1,...,2,12,0,0,0,1,0,0,53,95
6632,LAKSHADWEEP,TOTAL,2009,0,1,0,1,0,1,0,...,2,14,0,0,0,0,0,0,58,134
7411,LAKSHADWEEP,TOTAL,2010,0,0,0,0,0,0,0,...,2,1,0,0,0,0,0,0,26,42


In [43]:
# Filtering out the district "Total" from the dataframe. 

t_filter = crime_data['DISTRICT'] == 'TOTAL'
new_sample = crime_data[t_filter]

In [642]:
# Grouping should give exactly 12 counts for each state. Depending on that, we will do more clean up for the frame.

new_sample[['STATE/UT', 'YEAR']].groupby('STATE/UT').count().sort_values('YEAR', ascending=False)

,YEAR
STATE/UT,
A & N ISLANDS,12
PUDUCHERRY,12
MAHARASHTRA,12
MANIPUR,12
MEGHALAYA,12
MIZORAM,12
NAGALAND,12
ODISHA,12
PUNJAB,12


In [44]:
# Looking at indicators in the dataset.

list(crime_data)

['STATE/UT',
 'DISTRICT',
 'YEAR',
 'MURDER',
 'ATTEMPT TO MURDER',
 'CULPABLE HOMICIDE NOT AMOUNTING TO MURDER',
 'RAPE',
 'CUSTODIAL RAPE',
 'OTHER RAPE',
 'KIDNAPPING & ABDUCTION',
 'KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS',
 'KIDNAPPING AND ABDUCTION OF OTHERS',
 'DACOITY',
 'PREPARATION AND ASSEMBLY FOR DACOITY',
 'ROBBERY',
 'BURGLARY',
 'THEFT',
 'AUTO THEFT',
 'OTHER THEFT',
 'RIOTS',
 'CRIMINAL BREACH OF TRUST',
 'CHEATING',
 'COUNTERFIETING',
 'ARSON',
 'HURT/GREVIOUS HURT',
 'DOWRY DEATHS',
 'ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY',
 'INSULT TO MODESTY OF WOMEN',
 'CRUELTY BY HUSBAND OR HIS RELATIVES',
 'IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES',
 'CAUSING DEATH BY NEGLIGENCE',
 'OTHER IPC CRIMES',
 'TOTAL IPC CRIMES']

In [45]:
# for this project, let's start with "Murder" & "Rape" indicators. 

temp_crime_data = crime_data[t_filter]

t_crime_data = temp_crime_data[['STATE/UT','YEAR','MURDER','RAPE']]
t_crime_data.head()

,STATE/UT,YEAR,MURDER,RAPE
28,ANDHRA PRADESH,2001,2602,871
42,ARUNACHAL PRADESH,2001,83,33
70,ASSAM,2001,1356,817
115,BIHAR,2001,3643,888
135,CHHATTISGARH,2001,880,959


In [46]:
# exploring the data frame further to understand the value distribution, correlation.

print("Exploring values for the indicator - RAPE")
list1 = t_crime_data['RAPE'].unique().tolist()
print("Range --> Min Value : " + str(min(list1)) + " to " + str(max(list1)) )
print("Mean Value - " + str(t_crime_data['RAPE'].mean()))
print("Median Value - " + str(t_crime_data['RAPE'].median()))
print("Mode Value - " + str(t_crime_data['RAPE'].mode().values))
print("\n")
print("Exploring values for the indicator - MURDER")
list2 = t_crime_data['MURDER'].unique().tolist()
print("Range --> Min Value : " + str(min(list2)) + " to " + str(max(list2)) )
print("Mean Value - " + str(t_crime_data['MURDER'].mean()))
print("Median Value - " + str(t_crime_data['MURDER'].median()))
print("Mode Value - " + str(t_crime_data['MURDER'].mode().values))

Corr1 = t_crime_data['RAPE'].corr(temp_crime_data['ATTEMPT TO MURDER'])
print(Corr1)
Corr2 = t_crime_data['RAPE'].corr(temp_crime_data['ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY'])
print(Corr2)

Exploring values for the indicator - RAPE
Range --> Min Value : 0 to 3425
Mean Value - 570.375
Median Value - 281.5
Mode Value - [0 1]


Exploring values for the indicator - MURDER
Range --> Min Value : 0 to 7601
Mean Value - 971.4166666666666
Median Value - 381.0
Mode Value - [0]
0.68290373222
0.856239698998


In [47]:
t_crime_data.sort_values('STATE/UT')

,STATE/UT,YEAR,MURDER,RAPE
7384,A & N ISLANDS,2010,9,24
8986,A & N ISLANDS,2012,15,12
1409,A & N ISLANDS,2002,17,2
691,A & N ISLANDS,2001,13,3
5838,A & N ISLANDS,2008,8,12
5081,A & N ISLANDS,2007,15,3
2867,A & N ISLANDS,2004,15,10
8175,A & N ISLANDS,2011,14,13
3600,A & N ISLANDS,2005,14,4
4339,A & N ISLANDS,2006,4,6


In [48]:
# So, we have data for 12 years and to plot them let's take a mean of the values. 
# We are creating the marker values for the indicators to plot them in the map.

plot_crime_data = pd.DataFrame(t_crime_data[['STATE/UT', 'MURDER']].groupby('STATE/UT').mean())
plot_crime_data['Murder_Marker_Value'] = t_crime_data[['STATE/UT', 'MURDER']].groupby('STATE/UT').mean()/100
plot_crime_data['RAPE']=pd.DataFrame(t_crime_data[['STATE/UT', 'RAPE']].groupby('STATE/UT').mean())
plot_crime_data['Rape_Marker_Value'] = t_crime_data[['STATE/UT', 'RAPE']].groupby('STATE/UT').mean()/100
plot_crime_data['STATE/UT'] = t_crime_data['STATE/UT'].sort_values().unique()
plot_crime_data.head()

,MURDER,Murder_Marker_Value,RAPE,Rape_Marker_Value,STATE/UT
STATE/UT,,,,,
A & N ISLANDS,13.333333,0.133333,9.083333,0.090833,A & N ISLANDS
ANDHRA PRADESH,2646.333333,26.463333,1123.250000,11.232500,ANDHRA PRADESH
ARUNACHAL PRADESH,70.083333,0.700833,41.666667,0.416667,ARUNACHAL PRADESH
ASSAM,1286.000000,12.860000,1348.166667,13.481667,ASSAM
BIHAR,3437.083333,34.370833,1093.666667,10.936667,BIHAR


In [49]:
# Checking the dataset

t_crime_data.head()
#txt_f['place'] = pd.DataFrame(t_crime_data['STATE/UT'].sort_values().unique())
#txt_f.head()

,STATE/UT,YEAR,MURDER,RAPE
28,ANDHRA PRADESH,2001,2602,871
42,ARUNACHAL PRADESH,2001,83,33
70,ASSAM,2001,1356,817
115,BIHAR,2001,3643,888
135,CHHATTISGARH,2001,880,959


In [50]:
# Sorting the values to plot (data prep)

t_state = t_crime_data['STATE/UT'].unique()
t_state.sort()
t_state
state_list = t_state.tolist()

In [52]:
# We are getting latitude and longitude for all states to plot the indicators.

import time
#temp_list = list(National_Sites['Name'])
with open('./crime-in-india/external/state_ut_lat_lon.csv', 'a') as f:
    f.write("lat,lon,place")
    f.write('\n')
    for i in range(len(state_list)):
        if (temp_str == 'A & N ISLANDS'):
            temp_str = 'Andaman Nicobar Islands'
        else:
            temp_str
        temp_str = state_list[i].replace(" ", "+")
        line = str(get_lat_long(temp_str, state_list[i]))
        time.sleep(15)
        jk_line1 = line.replace("[","")
        jk_line2 = jk_line1.replace("]","")
        jk_line2 = jk_line2.replace("'","")
        jk_line2 = jk_line2.replace("'","")
        f.write(jk_line2)
        f.write('\n')

In [57]:
# checking the shapes of data frames, before we plot. 
    
#plot_crime_data.shape
t_final_frame = pd.read_csv('./crime-in-india/external/state_ut_lat_lon.csv')
t_final_frame.shape

(75, 3)

In [58]:
# Concatenating the data frames before plotting. 

plot_crime_data.reset_index(drop=True, inplace=True)
t_final_frame.reset_index(drop=True, inplace=True)

dp = pd.concat( [plot_crime_data, t_final_frame], axis=1)

In [59]:
dp.head()

,MURDER,Murder_Marker_Value,RAPE,Rape_Marker_Value,STATE/UT,lat,lon,place
0,13.333333,0.133333,9.083333,0.090833,A & N ISLANDS,11.740087,92.600000,A & N ISLANDS
1,2646.333333,26.463333,1123.250000,11.232500,ANDHRA PRADESH,15.912900,79.739987,ANDHRA PRADESH
2,70.083333,0.700833,41.666667,0.416667,ARUNACHAL PRADESH,28.217999,94.727753,ARUNACHAL PRADESH
3,1286.000000,12.860000,1348.166667,13.481667,ASSAM,26.200604,92.937574,ASSAM
4,3437.083333,34.370833,1093.666667,10.936667,BIHAR,25.096074,85.313119,BIHAR


In [60]:
# Plotting the indicators in bar to understand how the mean values are distributed

data = [Bar(x = dp.place, y = dp.RAPE, text=dp.RAPE, textposition = 'auto',opacity=0.6, )]
py.iplot(data, title='basic bar chart')

In [75]:
# Plotting the prominent spots along with Indicators

mapbox_access_token = plotly_creds['map_key']

df = pd.read_csv('./crime-in-india/external/np_lat_long.csv')
site_lat = df.lat
site_lon = df.lon
locations_name = df.place

#Indicator 1 - Murder
ind1_lat = dp.lat
ind1_lon = dp.lon
ind1_loc_name = "Indicator - MURDER"
ind1_sz = dp.Murder_Marker_Value

#Indicator 2 - Rape
ind2_lat = dp.lat
ind2_lon = dp.lon
ind2_loc_name = "Indicator - RAPE"
ind2_sz = dp.Rape_Marker_Value

data = Data([
    Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        name='Prominent Tourist Spots',
        marker=Marker(
            size=12,
            color='rgb(255, 0, 0)',
            opacity=0.8
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    Scattermapbox(
        lat=ind1_lat,
        lon=ind1_lon,
        mode='markers',
        name='Indicator - Murder',
        marker=Marker(
            size=ind1_sz+50,
            color='rgb(0, 0, 255)',
            opacity=0.4
        ),
        text=ind1_loc_name,
        hoverinfo='text'    
    ),
    Scattermapbox(
        lat=ind2_lat,
        lon=ind2_lon,
        mode='markers',
        name='Indicator - Rape',
        marker=Marker(
            size=ind2_sz+50,
            color='rgb(0, 102, 0)',
            opacity=0.5
        ),
        text=ind2_loc_name,
        hoverinfo='text'   
    )]
)
        
layout = Layout(
    title='Prominent Tourist Spots in India - Along with Indicators',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=22.6,
            lon=93.4
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='legend-names')

<B> This is an interactive map. More importantly, comparing this with the distribution (bar chart) gives more insight to understand how crimes are distributes across locations </B>

<h5 style="font-size:2em;color:#2467C0">Understanding Police Force Available</h5>

## Initial data exploration

In [99]:
pl_data = pd.read_csv('./crime-in-india/Crime_data/12_Police_strength_actual_and_sanctioned.csv')
pl_data.head()

,Area_Name,Year,Group_Name,Sub_Group_Name,Rank_All_Ranks_Total,Rank_ASI_Equivalent,Rank_ASPDySPAssttCommandant,Rank_Below_HC_and_Above_Constables,Rank_Constables,Rank_DGAddl_DG,Rank_DIG,Rank_Head_Constables,Rank_IGSplIG,Rank_Inspectors_Equivalent,Rank_SI_Equivalent,Rank_SSPSPAddlSPCommandant
0,Andaman & Nicobar Islands,2001,Actual Police Strength - Armed Police,A2. Acual Armed Police (Incl. Women Police),766,7,2,0,646,0,1,84,0,6,20,0
1,Andhra Pradesh,2001,Actual Police Strength - Armed Police,A2. Acual Armed Police (Incl. Women Police),12510,433,56,0,8742,0,0,2864,0,132,270,13
2,Arunachal Pradesh,2001,Actual Police Strength - Armed Police,A2. Acual Armed Police (Incl. Women Police),2232,14,15,169,1645,0,0,322,0,19,45,3
3,Assam,2001,Actual Police Strength - Armed Police,A2. Acual Armed Police (Incl. Women Police),23963,36,135,2347,16591,0,0,3868,0,235,699,52
4,Bihar,2001,Actual Police Strength - Armed Police,A2. Acual Armed Police (Incl. Women Police),373,0,0,0,326,0,0,41,0,2,4,0


In [100]:
# Let's understand the groups present. 
# For this project, we will select Actual Vs Sanctioned Police Strength

pl_data['Group_Name'].unique().tolist()

['Actual Police Strength - Armed Police',
 'Actual Police Strength - Civil Police',
 'Actual Police Strength - Total',
 'Actual Women Police Strength - Armed Police',
 'Actual Women Police Strength - Civil Police',
 'Actual Women Police Strength - Total',
 'Sanctioned Police Strength - Armed Police',
 'Sanctioned Police Strength - Civil Police',
 'Sanctioned Police Strength - Total',
 'Sanctioned Women Police Strength - Armed Police',
 'Sanctioned Women Police Strength - Civil Police',
 'Sanctioned Women Police Strength - Total']

In [101]:
list(pl_data)

['Area_Name',
 'Year',
 'Group_Name',
 'Sub_Group_Name',
 'Rank_All_Ranks_Total',
 'Rank_ASI_Equivalent',
 'Rank_ASPDySPAssttCommandant',
 'Rank_Below_HC_and_Above_Constables',
 'Rank_Constables',
 'Rank_DGAddl_DG',
 'Rank_DIG',
 'Rank_Head_Constables',
 'Rank_IGSplIG',
 'Rank_Inspectors_Equivalent',
 'Rank_SI_Equivalent',
 'Rank_SSPSPAddlSPCommandant']

In [102]:
pl_data.head()
pl_filter = pl_data['Group_Name'].str.contains('Total')
pl_filter.head()
pl_data.shape

(4188, 16)

In [103]:
n_frame = pl_data[pl_filter]
n_frame.head()
#n_frame.shape

,Area_Name,Year,Group_Name,Sub_Group_Name,Rank_All_Ranks_Total,Rank_ASI_Equivalent,Rank_ASPDySPAssttCommandant,Rank_Below_HC_and_Above_Constables,Rank_Constables,Rank_DGAddl_DG,Rank_DIG,Rank_Head_Constables,Rank_IGSplIG,Rank_Inspectors_Equivalent,Rank_SI_Equivalent,Rank_SSPSPAddlSPCommandant
698,Andaman & Nicobar Islands,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),2792,199,11,0,1976,0,2,437,1,31,134,1
699,Andhra Pradesh,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),77846,2906,353,0,56629,8,23,12875,27,1140,3757,128
700,Arunachal Pradesh,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),5398,157,35,169,3932,0,3,667,1,62,354,18
701,Assam,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),53345,1795,270,3238,38025,7,16,5664,11,722,3411,186
702,Bihar,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),45404,3309,156,259,32042,1,3,5018,0,333,4236,47


In [104]:
#n_frame['Rank_All_Ranks_Total'].unique().tolist()
n_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1396 entries, 698 to 4187
Data columns (total 16 columns):
Area_Name                             1396 non-null object
Year                                  1396 non-null int64
Group_Name                            1396 non-null object
Sub_Group_Name                        1396 non-null object
Rank_All_Ranks_Total                  1396 non-null int64
Rank_ASI_Equivalent                   1396 non-null int64
Rank_ASPDySPAssttCommandant           1396 non-null int64
Rank_Below_HC_and_Above_Constables    1396 non-null int64
Rank_Constables                       1396 non-null int64
Rank_DGAddl_DG                        1396 non-null int64
Rank_DIG                              1396 non-null int64
Rank_Head_Constables                  1396 non-null int64
Rank_IGSplIG                          1396 non-null int64
Rank_Inspectors_Equivalent            1396 non-null int64
Rank_SI_Equivalent                    1396 non-null int64
Rank_SSPSPAddlSPCo

In [105]:
# Creating actual data frames

act_filter = pl_data['Group_Name'].str.contains('Actual Police Strength')
sanct_filter = pl_data['Group_Name'].str.contains('Sanctioned Police Strength')

aframe = pd.DataFrame(n_frame[act_filter])
aframe.head()
#aframe.info()
#del aframe['Year']

/Users/anandkarthick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Area_Name,Year,Group_Name,Sub_Group_Name,Rank_All_Ranks_Total,Rank_ASI_Equivalent,Rank_ASPDySPAssttCommandant,Rank_Below_HC_and_Above_Constables,Rank_Constables,Rank_DGAddl_DG,Rank_DIG,Rank_Head_Constables,Rank_IGSplIG,Rank_Inspectors_Equivalent,Rank_SI_Equivalent,Rank_SSPSPAddlSPCommandant
698,Andaman & Nicobar Islands,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),2792,199,11,0,1976,0,2,437,1,31,134,1
699,Andhra Pradesh,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),77846,2906,353,0,56629,8,23,12875,27,1140,3757,128
700,Arunachal Pradesh,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),5398,157,35,169,3932,0,3,667,1,62,354,18
701,Assam,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),53345,1795,270,3238,38025,7,16,5664,11,722,3411,186
702,Bihar,2001,Actual Police Strength - Total,A3. Actual Police Strength (Incl. Women),45404,3309,156,259,32042,1,3,5018,0,333,4236,47


In [106]:
# Creating a frame with mean values for the indicator - "Actual Police Strength - Total"

a_frame = aframe.groupby('place').mean()
a_frame['place']=aframe['place'].unique().tolist()
a_frame.head()
a_frame['Actual_Mean'] = a_frame['Rank_All_Ranks_Total']
a_frame.head()

KeyError: 'place'

In [868]:
# Creating Sanctioned data frames

sanct_filter = pl_data['Group_Name'].str.contains('Sanctioned Police Strength')
sframe = n_frame[sanct_filter]
sframe.shape

(349, 3)

In [871]:
# Creating a frame with mean values for the indicator - "Sanctioned Police Strength - Total"

s_frame = sframe.groupby('place').mean()
s_frame['place']=sframe['place'].unique().tolist()
s_frame.head()
s_frame['Sanctioned_Mean'] = s_frame['Rank_All_Ranks_Total']
s_frame.head()

,Rank_All_Ranks_Total,place,Sanctioned_Mean
place,,,
Andaman & Nicobar Islands,3219.1,Andaman & Nicobar Islands,3219.1
Andhra Pradesh,93663.6,Andhra Pradesh,93663.6
Arunachal Pradesh,6610.0,Arunachal Pradesh,6610.0
Assam,55950.4,Assam,55950.4
Bihar,75673.3,Bihar,75673.3


In [913]:
#dx = pd.concat( [a_frame, s_frame], axis=1)
dx.head()

,place,Actual_Mean,place,Sanctioned_Mean
place,,,,
Andaman & Nicobar Islands,Andaman & Nicobar Islands,2971.4,Andaman & Nicobar Islands,3219.1
Andhra Pradesh,Andhra Pradesh,84085.6,Andhra Pradesh,93663.6
Arunachal Pradesh,Arunachal Pradesh,6146.8,Arunachal Pradesh,6610.0
Assam,Assam,52443.1,Assam,55950.4
Bihar,Bihar,54630.9,Bihar,75673.3


In [937]:
dx['Occupancy(%)'] = (dx['Actual_Mean']/dx['Sanctioned_Mean'])*100
dx['Vacancy(%)'] = 100 - dx['Occupancy(%)']
dx.head()

,place,Actual_Mean,Sanctioned_Mean,Occupancy(%),Vacancy(%)
place,,,,,
Andaman & Nicobar Islands,Andaman & Nicobar Islands,2971.4,3219.1,92.3053,7.6947
Andhra Pradesh,Andhra Pradesh,84085.6,93663.6,89.774,10.226
Arunachal Pradesh,Arunachal Pradesh,6146.8,6610,92.9924,7.00756
Assam,Assam,52443.1,55950.4,93.7314,6.26859
Bihar,Bihar,54630.9,75673.3,72.1931,27.8069


In [942]:
dx = dx.sort_values('place')
dx

,place,Actual_Mean,Sanctioned_Mean,Occupancy(%),Vacancy(%)
place,,,,,
Andaman & Nicobar Islands,Andaman & Nicobar Islands,2971.4,3219.1,92.3053,7.6947
Andhra Pradesh,Andhra Pradesh,84085.6,93663.6,89.774,10.226
Arunachal Pradesh,Arunachal Pradesh,6146.8,6610,92.9924,7.00756
Assam,Assam,52443.1,55950.4,93.7314,6.26859
Bihar,Bihar,54630.9,75673.3,72.1931,27.8069
Chandigarh,Chandigarh,4181.2,4710.6,88.7615,11.2385
Chhattisgarh,Chhattisgarh,26455.4,31085.4,85.1055,14.8945
Dadra & Nagar Haveli,Dadra & Nagar Haveli,219.9,244.2,90.0491,9.95086
Daman & Diu,Daman & Diu,231.5,242.8,95.346,4.65404


In [95]:
# Plotting the trend using plotly

val_x = dx.place
act_y = dx.Actual_Mean
sct_y = dx.Sanctioned_Mean
trace = Scatter(
    x = val_x,
    y = act_y,
    mode = 'lines+markers',
    name = 'Actual')
trace1 = Scatter(
    x = val_x,
    y = sct_y,
    mode = 'lines+markers',
    name = 'Sanctioned')

data = [trace, trace1]
layout = dict(title='Actual vs Sanctioned Police Force (mean value)')
fig = dict(data=data, layout=layout)
#trace = Scatter(x,y)
py.iplot(fig,file_name='line')



NameError: name 'dx' is not defined